# Train the model to colorize images. 

#Put images in a certain directory. 
## Prepare 6 train images and test images.

In [0]:
path = '/content/img'
train_datagen = ImageDataGenerator(rescale=1. / 255)

In [0]:
import glob
import PIL
from PIL import Image
train_img = []
train_dir = r'/content/img'
for im in glob.glob(train_dir+'/*.jpg'):
#for im in tr_data:
    im1 = Image.open(im)
    im = im1.resize((256,256),PIL.Image.ANTIALIAS)
    train_img.append(im)
print(train_img)
print(len(train_img))

[<PIL.Image.Image image mode=RGB size=256x256 at 0x7F036276FE80>, <PIL.Image.Image image mode=RGB size=256x256 at 0x7F03627C8D68>, <PIL.Image.Image image mode=RGB size=256x256 at 0x7F036269DEF0>, <PIL.Image.Image image mode=RGB size=256x256 at 0x7F036269DCF8>, <PIL.Image.Image image mode=RGB size=256x256 at 0x7F036269DC88>, <PIL.Image.Image image mode=RGB size=256x256 at 0x7F036269DCC0>]
6


In [0]:
test_img=[]
test_dir = r'/content/test_img'
for im in glob.glob(test_dir+'/*.jpg'):
#for im in tr_data:
    im1 = Image.open(im)
    im = im1.resize((256,256),PIL.Image.ANTIALIAS)
    test_img.append(im)
X_test=[]
for img in train_img:
    lab = rgb2lab(img)
    X_test.append(lab[:,:,0])
#X_test = X_test.reshape(X_test.shape+(1,))
print(len(X_test))
print(X_test[1])
print(X_test[1].shape)

6
[[35.24402364 36.2487985  38.9548529  ... 37.51177801 37.07615668
  36.40400918]
 [35.65282586 37.47198985 40.12333705 ... 37.73252611 37.2929758
  36.7605465 ]
 [36.06109832 38.3327681  40.66366364 ... 37.91139517 37.69211843
  37.33396655]
 ...
 [19.96856806 19.6378602  19.34592902 ... 40.81491762 40.07095071
  39.88901756]
 [20.23468874 19.56856679 19.73988082 ... 40.55245438 40.47583721
  40.44929125]
 [20.18138154 19.96856806 19.99836765 ... 39.96476613 39.81140489
  39.75085018]]
(256, 256)


In [0]:
X =[]
Y =[]
for img in train_img:
  try:
      lab = rgb2lab(img)
      X.append(lab[:,:,0])
      Y.append(lab[:,:,1:] / 128)
  except:
     print('error')
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,))
print(X.shape)
print(Y.shape)

(6, 256, 256, 1)
(6, 256, 256, 2)


#Training the model

In [0]:
session = tf.Session()
x = tf.placeholder(tf.float32, shape = [None, 256, 256, 1], name = 'x')
#x=Input(shape=(256,256,1))
ytrue = tf.placeholder(tf.float32, shape = [None, 256, 256, 2], name = 'ytrue')
def create_weights(shape):
  return tf.Variable(tf.truncated_normal(shape, stddev=0.1))
def create_bias(size):
  return tf.Variable(tf.constant(0.1, shape = [size]))
def convolution(inputs, num_channels, filter_size, num_filters):
  weights = create_weights(shape = [filter_size, filter_size, num_channels, num_filters])
  bias = create_bias(num_filters)
  
  ## convolutional layer
  layer = tf.nn.conv2d(input = inputs, filter = weights, strides= [1, 1, 1, 1], padding = 'SAME') + bias
  layer = tf.nn.tanh(layer)
  return layer
def maxpool(inputs, kernel, stride):
  layer = tf.nn.max_pool(value = inputs, ksize = [1, kernel, kernel, 1], strides = [1, stride, stride, 1], padding = "SAME")
  return layer
def upsampling(inputs):
  layer = tf.image.resize_nearest_neighbor(inputs, (2*inputs.get_shape().as_list()[1], 2*inputs.get_shape().as_list()[2]))
  return layer
conv1 = convolution(x, 1, 3, 3)
max1 = maxpool(conv1, 2, 2)
conv2 = convolution(max1, 3, 3, 8)
max2 = maxpool(conv2, 2, 2)
conv3 = convolution(max2, 8, 3, 16)
max3 = maxpool(conv3, 2, 2)
conv4 = convolution(max3, 16, 3, 16)
max4 = maxpool(conv4, 2, 2)
conv5 = convolution(max4, 16, 3, 32)
max5 = maxpool(conv5, 2, 2)
conv6 = convolution(max5, 32, 3, 32)
max6 = maxpool(conv6, 2, 2)
conv7 = convolution(max6, 32, 3, 64)
upsample1 = upsampling(conv7)
conv8 = convolution(upsample1, 64, 3, 32)
upsample2 = upsampling(conv8)
conv9 = convolution(upsample2, 32, 3, 32)
upsample3 = upsampling(conv9)
conv10 = convolution(upsample3, 32, 3, 16)
upsample4 = upsampling(conv10)
conv11 = convolution(upsample4, 16, 3, 16)
upsample5 = upsampling(conv11)
conv12 = convolution(upsample5, 16, 3, 8)
upsample6 = upsampling(conv12)
conv13 = convolution(upsample6, 8, 3, 2)
loss = tf.losses.mean_squared_error(labels = ytrue, predictions = conv13)
cost = tf.reduce_mean(loss)
optimizer = tf.train.AdamOptimizer(learning_rate = 0.0001).minimize(cost)
session.run(tf.global_variables_initializer())

In [0]:
num_epochs = 1500
for i in range(num_epochs):
    session.run(optimizer, feed_dict = {x: X, ytrue:Y})
    lossvalue = session.run(cost, feed_dict = {x:X, ytrue : Y})
    print("epoch: " + str(i) + " loss: " + str(lossvalue))

epoch: 0 loss: 0.07058012
epoch: 1 loss: 0.05880915
epoch: 2 loss: 0.049540598
epoch: 3 loss: 0.043145966
epoch: 4 loss: 0.038944565
epoch: 5 loss: 0.035608064
epoch: 6 loss: 0.0327073
epoch: 7 loss: 0.0304252
epoch: 8 loss: 0.028793657
epoch: 9 loss: 0.027644327
epoch: 10 loss: 0.02680426
epoch: 11 loss: 0.026173228
epoch: 12 loss: 0.02569477
epoch: 13 loss: 0.025318988
epoch: 14 loss: 0.024983803
epoch: 15 loss: 0.02462787
epoch: 16 loss: 0.024223728
epoch: 17 loss: 0.023782002
epoch: 18 loss: 0.023339478
epoch: 19 loss: 0.022935515
epoch: 20 loss: 0.022594886
epoch: 21 loss: 0.022320686
epoch: 22 loss: 0.022097034
epoch: 23 loss: 0.021901665
epoch: 24 loss: 0.021717349
epoch: 25 loss: 0.021538682
epoch: 26 loss: 0.021370025
epoch: 27 loss: 0.02121815
epoch: 28 loss: 0.021083599
epoch: 29 loss: 0.020961
epoch: 30 loss: 0.02084245
epoch: 31 loss: 0.02072337
epoch: 32 loss: 0.020604584
epoch: 33 loss: 0.020491255
epoch: 34 loss: 0.020389153
epoch: 35 loss: 0.02030108
epoch: 36 loss: 0.

In [0]:
import numpy as np
import cv2
import tensorflow as tf
import skimage.color as color
import skimage.io as io
import os
test_path = '/content/test_img'
img = img_to_array(load_img(test_path+'/'+str(1)+'.png'))
img = resize(img,(256,256))
print(img.shape)

(256, 256, 3)


In [0]:
output = session.run(conv13, feed_dict = {x: X[4].reshape([1, 256, 256, 1])})*128
image = np.zeros([256, 256, 3])
image[:,:,0]=X[0][:,:,0]
image[:,:,1:]=output[0]
image = color.lab2rgb(image)
io.imsave("test_img6.jpg", image)

In [0]:

test_path = '/content/test_img'
for i in range(1,6):
  color_me = []
  img = img_to_array(load_img(test_path+'/'+str(i)+'.png'))
  img = resize(img,(256,256))
  color_me.append(img)
  color_me = np.array(color_me, dtype=float)
  color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
  color_me = color_me.reshape(color_me.shape+(1,))
  output = session.run(conv13, feed_dict = {x: X[0].reshape([1, 256, 256, 1])})*128
  image = np.zeros([256, 256, 3])
  image[:,:,0]=X[0][:,:,0]
  image[:,:,1:]=output[0]
  '''output = output * 128
  result = np.zeros((256, 256, 3))
  result[:,:,0] = color_me[0][:,:,0]
  result[:,:,1:] = output[0]'''
  imsave('img_model2_'+str(i+1)+'.png',lab2rgb(result))